In [1]:
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import ss.implicits._
import org.apache.spark.sql.functions._
import java.nio.file.Paths

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/12/11 17:17:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Compiling /home/eczech/repos/ot-scoring/notebooks/pathinit.sc

import $file.$          , sparkinit._

import $file.$         , pathinit._

import ss.implicits._

import org.apache.spark.sql.functions._

import java.nio.file.Paths

In [2]:
val input_dir = Paths.get(EXTRACT_DIR.resolve("evidence_raw.json").toString)
val df = ss.read.json(input_dir.toString)
df

json at cmd1.sc:2

9 / 9

19/12/11 17:18:15 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


input_dir: java.nio.file.Path = /home/eczech/data/ot/extract/evidence_raw.json
df: org.apache.spark.sql.package.DataFrame = [_corrupt_record: string, access_level: string ... 10 more fields]
res1_2: org.apache.spark.sql.package.DataFrame = [_corrupt_record: string, access_level: string ... 10 more fields]

In [3]:
df.select("disease.id").show(10, truncate=false)

show at cmd2.sc:1

1 / 1

+-----------------------------------------+
|id                                       |
+-----------------------------------------+
|http://www.orpha.net/ORDO/Orphanet_1572  |
|http://www.orpha.net/ORDO/Orphanet_1572  |
|http://www.orpha.net/ORDO/Orphanet_183660|
|http://www.orpha.net/ORDO/Orphanet_157949|
|http://www.orpha.net/ORDO/Orphanet_935   |
|http://www.orpha.net/ORDO/Orphanet_300298|
|http://www.orpha.net/ORDO/Orphanet_2141  |
|http://www.orpha.net/ORDO/Orphanet_229717|
|http://www.orpha.net/ORDO/Orphanet_2856  |
|http://www.orpha.net/ORDO/Orphanet_399808|
+-----------------------------------------+
only showing top 10 rows



In [5]:
val dfp = df.withColumn("m", split($"disease.id", "/")).select(
    $"m".getItem(2).as("domain"),
    $"m".getItem(3).as("dir"),
    $"m".getItem(4).as("id")
)
dfp.groupBy("domain", "dir").count().show

show at cmd4.sc:6

9 / 9

show at cmd4.sc:6

1 / 1

+-------------------+----+-------+
|             domain| dir|  count|
+-------------------+----+-------+
|      www.orpha.net|ORDO| 617976|
|      www.ebi.ac.uk| efo|1050929|
|purl.obolibrary.org| obo|  61662|
|               null|null|      1|
+-------------------+----+-------+



dfp: org.apache.spark.sql.package.DataFrame = [domain: string, dir: string ... 1 more field]

In [7]:
dfp.filter($"dir" === "ORDO").show(3, truncate=false)

show at cmd6.sc:1

1 / 1

+-------------+----+---------------+
|domain       |dir |id             |
+-------------+----+---------------+
|www.orpha.net|ORDO|Orphanet_1572  |
|www.orpha.net|ORDO|Orphanet_1572  |
|www.orpha.net|ORDO|Orphanet_183660|
+-------------+----+---------------+
only showing top 3 rows



In [8]:
dfp.filter($"dir" === "efo").show(3, truncate=false)

show at cmd7.sc:1

1 / 1

+-------------+---+-----------+
|domain       |dir|id         |
+-------------+---+-----------+
|www.ebi.ac.uk|efo|EFO_0005571|
|www.ebi.ac.uk|efo|EFO_1001500|
|www.ebi.ac.uk|efo|EFO_0005571|
+-------------+---+-----------+
only showing top 3 rows



In [9]:
dfp.filter($"dir" === "obo").show(3, truncate=false)

show at cmd8.sc:1

1 / 1

+-------------------+---+----------+
|domain             |dir|id        |
+-------------------+---+----------+
|purl.obolibrary.org|obo|HP_0000175|
|purl.obolibrary.org|obo|HP_0001943|
|purl.obolibrary.org|obo|HP_0004936|
+-------------------+---+----------+
only showing top 3 rows



In [11]:
df.withColumn("m", split($"disease.id", "/"))
    .filter(($"m".getItem(3) === "efo"))
    .groupBy($"m".getItem(4).startsWith("EFO"))
    .count().show

show at cmd10.sc:3

9 / 9

show at cmd10.sc:3

1 / 1

+---------------------+-------+
|startswith(m[4], EFO)|  count|
+---------------------+-------+
|                 true|1050929|
+---------------------+-------+



In [10]:
df.withColumn("m", split($"disease.id", "/"))
    .filter(($"m".getItem(3) === "ORDO"))
    .groupBy($"m".getItem(4).startsWith("Orphanet"))
    .count().show

show at cmd9.sc:3

9 / 9

show at cmd9.sc:3

1 / 1

+--------------------------+------+
|startswith(m[4], Orphanet)| count|
+--------------------------+------+
|                      true|617976|
+--------------------------+------+



In [13]:
df.withColumn("m", split($"disease.id", "/"))
    .filter(($"m".getItem(3) === "obo"))
    .groupBy($"m".getItem(4).startsWith("HP"))
    .count().show

show at cmd12.sc:3

9 / 9

show at cmd12.sc:3

1 / 1

+--------------------+-----+
|startswith(m[4], HP)|count|
+--------------------+-----+
|                true|42922|
|               false|18740|
+--------------------+-----+



In [17]:
df.withColumn("m", split($"disease.id", "/"))
    .filter(($"m".getItem(3) === "obo"))
    .withColumn("p", element_at(split($"m".getItem(4), "_"), 1))
    .groupBy("p").count.show

show at cmd16.sc:4

9 / 9

show at cmd16.sc:4

1 / 1

+------+-----+
|     p|count|
+------+-----+
|    HP|42922|
|    MP| 1452|
|    GO| 6151|
|  DOID|  258|
|  NCIT|   40|
| MONDO|10827|
|UBERON|   12|
+------+-----+

